In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
from fastdtw import fastdtw
from sklearn.preprocessing import normalize

# extract x, y from skeleton & change to (640 * 720)

In [2]:
def origin_rate(df):
    df_xy = df[[(a,b) for a,b in df.columns if b in ('x', 'y')]]
    df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
    df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
    return df_xy

# bounding box & perspective transform

In [3]:
def box_perspective(df_xy):
    for idx in range(len(df_xy)):
        x_min = df_xy.loc[idx, (slice(None), 'x')].min()
        x_max = df_xy.loc[idx, (slice(None), 'x')].max()
        y_min = df_xy.loc[idx, (slice(None), 'y')].min()
        y_max = df_xy.loc[idx, (slice(None), 'y')].max()
        
        if x_max != 0 and y_max != 0:
            x_rate = 640/((x_max+20)-(x_min-20))
            y_rate = 720/((y_max+20)-(y_min-20))

            df_xy.loc[idx, (slice(None), 'x')] = df_xy.loc[idx, (slice(None), 'x')].apply(lambda x : ((x-(x_min-20))*x_rate)/640)
            df_xy.loc[idx, (slice(None), 'y')] = df_xy.loc[idx, (slice(None), 'y')].apply(lambda x : ((x-(y_min-20))*y_rate)/720)
        
    return df_xy

# transform to eigen vector

In [4]:
def vector_transform(joint):    
    # Compute angles between joints
    v1 = joint[[13, 23, 14, 24, 15, 16, 17, 19, 21, 18, 20, 22, 25, 26, 27, 28, 29, 31, 30, 32], :] # Parent joint
    v2 = joint[[11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 16, 23, 24, 25, 26, 27, 27, 28, 28], :] # Child joint
    v = v2 - v1 # [20, 3]
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    
    return v

# cosine similarity

In [5]:
# scoring with real dancing datadef cosine_similarity(train, test):
    cossim_list = []
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        cossim_list.append(cossim)
    return np.nanmean(cossim_list)

# score by cosine similarity

In [6]:
def cossim_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    #print(train.shape, test.shape)
    
    train = box_perspective(train)
    test = box_perspective(test)
    #print(train.shape, test.shape)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)
        #print(train_joint)
        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)
        
        score = cosine_similarity(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    
    return np.nanmean(score_list)

# euclidean distance of cosine similarity

In [7]:
def euclid_distance(train, test):
    euclid_list = []
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        euc_d = np.sqrt(2*(1-cossim))
        score = 100 - (100 * (euc_d)/2)
        euclid_list.append(score)
    return np.nanmean(euclid_list)

# score by euclidean distance of cosine similarity

In [8]:
def euclid_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)

        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)

        score = euclid_distance(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    
    return np.nanmean(score_list)

# fastdtw

In [9]:
def fast_dtw(train, test):
    if np.isnan(train).any():
        return np.nan
    
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
        
    return fastdtw(train,test)[0]

# score by fastdtw

In [10]:
def fastdtw_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)

        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)
        #print(train_v, train_v.shape, '*'*100, test_v, test_v.shape, sep='\n')
        score = fast_dtw(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    return np.nanmean(score_list)

# load train, test data

In [11]:
verse3 = pd.read_csv("./csv/verse3_skeleton.csv", header=[0,1])

# scoring with real dancing data

In [12]:
cam = pd.read_csv("./csv/cam_skeleton.csv", header=[0,1])

### 1. cosine similarity

In [13]:
cossim_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.6219454735029377, 0.5677139128674856, 0.48397201058491746, 0.38885257786949123, 0.30959866505822775, 0.24684845378133588, 0.2119117535764541, 0.25807769146080395, 0.2556865037281176, 0.4409262283719375, 0.5416099246553614, 0.6763136488174701, 0.9462364106192792, 0.9571350427797854, 0.9572334125986128, 0.9578062800054745, 0.9567840426637476, 0.9544940122876092, 0.9490397839297573, 0.9507717064142372, 0.9541266658020634, 0.954687341894861, 0.8101690298768347, 0.7261437157683411, 0.6887248460932788, 0.6947494986573307, 0.6910107082116258, 0.7026434380289794, 0.7026445587185841, 0.6840942355312034, 0.6549803514182351, 0.6472581295125955, 0.6541882258630227, 0.6503665129821502, 0.7250912712099344, 0.6545224945047087, 0.5985425200347921, 0.57600152

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3788341648.py:6: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.745572862280136

- 0.745572862280136

### 2. fastdtw

In [14]:
fastdtw_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.430756217330257, 15.322180513964437, 18.024125047259183, 20.660850980360223, 22.60122668250306, 24.504842676008124, 24.87070885038197, 24.79432795435126, 23.19990483251869, 12.983962647316435, 14.848904517856194, 12.844466530384782, 5.641148989412879, 4.560089583675541, 4.750035154115668, 4.638517919674517, 4.608646163862909, 4.825069309510306, 5.687108699148602, 5.550880073601771, 5.209216195608143, 5.5585223784663, 10.151798997248767, 12.242165187587698, 12.953225052529694, 12.995928776254702, 12.725958210012951, 12.531484041813348, 12.025455663049234, 12.087933982162443, 12.422808265332021, 12.588182222341315, 12.223167649595416, 12.354216174384474, 10.534517124244713, 12.025523235782671, 14.12068563810232, 16.40569064547065, 15.330300196

11.294097344184873

- 11.294097344184873

### 3. euclidean distance of cosine similarity

In [15]:
euclid_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 65.5469463151575, 63.02140548165187, 59.27260799908457, 55.250125383652836, 52.23339564946586, 49.952855863468, 48.80544847298571, 51.02154851503025, 51.783563165231385, 59.341934352904616, 64.09632743749877, 71.57613097541709, 88.62340307988087, 90.9314911257064, 90.41270597214185, 90.77001527388616, 90.93821216379574, 89.93834965678361, 88.48048485027877, 88.51494127399081, 89.33656888334626, 89.05278760961268, 78.35459146304427, 74.32855967682642, 72.86254683573323, 73.20200842755017, 73.1886001003937, 73.91283299638539, 74.86973188603147, 74.19292757166725, 73.20850389180936, 72.21285298448689, 72.20628172831607, 71.65250657233796, 74.89480247974976, 70.91517212029092, 67.63176382035402, 65.62458572098365, 69.82497479864739, 72.549150507651

75.10838243657123

- 75.10838243657123

# scoring with different data

In [16]:
mjbj = pd.read_csv("./csv/mjbj2_skeleton.csv", header=[0,1])

### 1. cosine similarity

In [17]:
cossim_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 0.6454375475599744, 0.6340812362744941, 0.656844783756015, 0.6697085531016291, 0.6680984376402617, 0.6940112068772611, 0.7104330408029267, 0.7495546969532088, 0.7546706198865308, 0.7647886854087822, 0.7606217412957387, 0.7392201344569734, 0.7231787203130571, 0.7345776609624523, 0.7341430149365306, 0.7451910387101963, 0.7379135052347037, 0.7589177586355075, 0.7842291882750108, 0.7668506016647865, 0.7513374949291349, 0.7492223828755218, 0.7513879272002939, 0.7566797256550861, 0.7817775727805207, 0.7964834174193017, 0.7381789606453196, 0.7192053224690522, 0.6968902201467015, 0.711590861447899, 0.7112842384988985, 0.7438687116946292, 0.7552717769746529, 0.7393635279141841, 0.7411056268884292, 0.739065884816134, 0.7466977861710584, 0.7481433100045577, 0.7432130090056537, 0.7274757206923466, 0.7096529792109626, 0.6967798023196103, 0.689983082497496, 0.6741091372764875, 0.6449985778342404, 0.644541490510754, 0.6497092096374169, 0.6633638267123907, 0.6672795532995661, 0.6693611663057357,

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3788341648.py:6: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.4477193628349232

- 0.4477193628349232

### 2. fastdtw

In [18]:
fastdtw_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 12.138315389374764, 14.971500668111085, 13.913553283823935, 13.98379198282624, 11.39056577219457, 10.77067397370989, 10.541811318435867, 9.435515079603881, 11.382981156255488, 10.195701959304754, 10.958024170107656, 11.661504412177365, 11.628350015540496, 11.236192653445991, 10.701003952719631, 11.171880328519809, 11.451965138582613, 10.471432212661314, 10.351781380118723, 11.47782598457483, 11.674138244833305, 11.459936895717856, 11.589965985365312, 11.46900561024373, 11.170195924955358, 11.36812842927198, 11.452770266876001, 11.252664257156013, 11.571155836077011, 11.607743475448801, 11.746058347384025, 11.528849716380716, 9.98542356637555, 11.542574405502673, 11.5635346091333, 11.711612007196207, 10.25138855701679, 10.063136982654687, 11.372042963937556, 11.292720595251645, 11.43693727422896, 10.813780454689967, 10.419074131068014, 10.428866285306665, 9.917036960893348, 9.538474914596476, 9.238724033035497, 9.288460079706994, 9.123012203604189, 9.03958017401803, 9.109941173987

17.005956560926318

- 17.005956560926318

### 3. euclidean distance of cosine similarity

In [19]:
euclid_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 65.69151364976774, 65.20729505246469, 66.41027552134794, 66.71580907792823, 66.27544554223996, 68.66445076840924, 69.49189335779997, 71.94464196626066, 72.54921567561331, 73.52218014133635, 72.6659762473575, 71.94403669552347, 72.02113610611946, 71.8992659060074, 70.56958852255303, 73.09103381734096, 72.19143227482431, 73.18707399094201, 74.81852004918385, 74.50911474255986, 73.07738060762514, 73.07740952027693, 73.56488059966271, 73.8449315563233, 74.83879969035277, 75.73497340508635, 72.89521959975107, 71.92770130485073, 70.35890235216387, 70.67283340517001, 70.80179393246507, 72.26136076841021, 73.19837149620476, 72.32026148680328, 72.34906777528937, 72.19919167665664, 72.404417921816, 72.58983027042481, 72.7184393786666, 72.40545113841283, 71.6069859516015, 70.41024085918393, 70.44033638768207, 69.95254763826767, 68.27756315480428, 68.60294606017173, 68.73127965699206, 69.37644866829774, 69.64879218054243, 69.84571452637685, 69.99723523872375, 70.04836100058928, 69.5653496134

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3788341648.py:6: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\2344704103.py:10: RuntimeWarning: Mean of empty slice
  return np.nanmean(euclid_list)


58.25387908688379

- 58.25387908688379

# scoring with null data

In [20]:
df_nan = pd.DataFrame(np.full((382,132), np.nan), columns=cam.columns)

In [21]:
df_nan.head()

NOSE             LEFT_EYE_INNER             LEFT_EYE      ... RIGHT_HEEL  \
     x   y   z   v              x   y   z   v        x   y  ...          z   
0  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
1  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
2  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
3  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
4  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   

      LEFT_FOOT_INDEX             RIGHT_FOOT_INDEX              
    v               x   y   z   v                x   y   z   v  
0 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
1 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
2 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
3 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
4 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  

[5 rows x 132 columns]

### 1. cosine similarity

In [22]:
cossim_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3788341648.py:6: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.0

- 0.0

### 2. fastdtw

In [41]:
fastdtw_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]


0.0

- 0.0

### 3. euclidean distance of cosine similarity

In [24]:
euclid_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\3788341648.py:6: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_26704\2344704103.py:10: RuntimeWarning: Mean of empty slice
  return np.nanmean(euclid_list)


0.0

- 0.0

# 정리

### 실제 동일하게 춤을 춘 경우

- cosine similarity : 0.745572862280136
- fastdtw : 11.294097344184873
- euclidean distance of cosine similarity : 75.10838243657123

### 다른 동작일 경우

- cosine similarity : 0.4477193628349232
- fastdtw : 17.005956560926318
- euclidean distance of cosine similarity : 58.25387908688379

### null data인 경우

- cosine similarity, fastdtw, euclidean distance of similarity : 모두 0.0

# 결론

- 벡터의 개수를 늘린 결과
- 다른 동작일 때 점수를 더 낮게 도출시킴
- 그렇지만 같은 동작일 때도 점수가 낮아짐